In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
%%time
import lib.utils.utils as utils
from run import get_images
#import lib.config as config
from lib.model.base_model import BaseModel
#import lib.roi as roi
#import lib.split_pages as spages
from lib.utils.promptLoader import PromptLoader

CPU times: user 355 ms, sys: 362 ms, total: 717 ms
Wall time: 1min 18s


In [3]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg"
image_dir = "images/cropped/"
default_yaml = "./prompts/default_v2_fromtext.yaml"
conversation = None

In [4]:
print(image_path)

images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg


In [5]:
images = sorted(utils.load_images(image_dir))[:2]

In [6]:
model = BaseModel("qwen_model", prompt=default_yaml, conversation=conversation, max_new_tokens = 130000, batch_size=1, temperature=1, save_path="./outputs/lightfootcat/")

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 5/5 [06:31<00:00, 78.26s/it]


In [7]:
outputs = model(images, save=True, debug=False, save_file_name="two_images")

Model: qwen_model | Batch Size: 1, Max Tokens: 130000, Temperature: 1
Extracting Text from Images


Processing Batches: 100%|██████████| 2/2 [01:51<00:00, 55.72s/batch]


Converting extracted text into Text Blocks
Organising text into JSON blocks


Processing Families in Dicotyledones: 100%|██████████| 9/9 [01:06<00:00,  7.42s/family]


Dumping extracted text into ./outputs/lightfootcat/two_images.json
